In [ ]:
import phasenet.zernike as Z
import phasenet.psf as P
import sys
sys.path.append('./../oldPack/')
import MultiIndicesTree
import Solvers

In [ ]:
def chebpoints(n):
    return np.cos(np.arange(n,dtype=np.float128)*np.pi/(n-1))

In [ ]:
def chebpoints_first(n):
    return np.cos(((2*np.arange(1,n+2,dtype=np.float128)-1) *np.pi)/(2*(n+1)))

In [ ]:
def spherical_ordering(n):
    arr = chebpoints_first(n)
    arr_even = np.sort(arr[arr>=0])
    arr_odd = arr[arr<0]
    new_arr = []
    for i,a in enumerate(arr_even):
        new_arr.append(a)
        try:
            new_arr.append(arr_odd[i])
        except:
            pass
    new_arr = np.expand_dims(new_arr,0)
    return new_arr

In [ ]:
def Leja(n):
    # Leja Ordering Chebyshev nodes
    points = chebpoints(n + 1)[::-1]
    Points = points
    ord = np.arange(1, n + 1)

    LJ = np.zeros([1, n + 1])
    LJ[0] = 0
    M = 0

    for k in range(0, n):
        JJ = 0
        for i in range(0, n - k):
            P = 1
            for j in range(k + 1):
                idx_pts = int(LJ[0, j])
                P = P * (points[idx_pts] - points[ord[i]])
            P = np.abs(P)
            if (P >= M):
                JJ = i
                M = P
        M = 0
        LJ[0, k + 1] = ord[JJ]
        ord = np.delete(ord, JJ)

    Leja_Points = np.zeros([1, n + 1])
    for i in range(n + 1):
        Leja_Points[0, i] = Points[int(LJ[0, i])]
    return Leja_Points

In [ ]:
def gen_points_lp(m, N, Points, Gamma):
    PP = np.zeros((m, N))
    for i in range(N):
        for j in range(m):
            PP[j,i] = Points[j, int(Gamma[j, i])]
    return PP

In [ ]:
def Gamma_lp(m, n, gamma, gamma2, p):
        gamma0 = gamma.copy()
        gamma0[m - 1] = gamma0[m - 1] + 1

        norm = np.linalg.norm(gamma0.reshape(-1), p)
        if (norm < n and m > 1):
            o1 = Gamma_lp(m - 1, n, gamma.copy(), gamma.copy(), p)
            o2 = Gamma_lp(m, n, gamma0.copy(), gamma0.copy(), p)
            out = np.concatenate([o1, o2], axis=-1)
        elif (norm < n and m == 1):
            out = np.concatenate([gamma2, Gamma_lp(m, n, gamma0.copy(), gamma0.copy(), p)], axis=-1)
        elif (norm == n and m > 1):
            out = np.concatenate([Gamma_lp(m - 1, n, gamma.copy(), gamma.copy(), p), gamma0], axis=-1)
        elif (norm == n and m == 1):
            out = np.concatenate([gamma2, gamma0], axis=-1)
        elif (norm > n):
            norm_ = np.linalg.norm(gamma.reshape(-1), p)
            if (norm_ < n and m > 1):
                for j in range(1, m):
                    gamma0 = gamma.copy()
                    gamma0[j - 1] = gamma0[j - 1] + 1  # gamm0 -> 1121 broken
                    if (np.linalg.norm(gamma0.reshape(-1), p) <= n):
                        gamma2 = np.concatenate([gamma2, Gamma_lp(j, n, gamma0.copy(), gamma0.copy(), p)], axis=-1)
                out = gamma2
            elif (m == 1):
                out = gamma2
            elif (norm_ <= n):
                out = gamma
            else:
                out = []

        return out

In [ ]:
m = 6 #Dimensions
n = 2 #degree
p = 5

gamma = np.zeros((m, 1))
Gamma = Gamma_lp(m, n, gamma, gamma.copy(), p)
N = Gamma.shape[1]

GP = np.zeros((m, n + 1))

# for i in range(m):
#     GP[i,] = (-1) ** (i + 1) * Leja(n)

for i in range(m):
    GP[i,] = (-1) ** (i + 1) * spherical_ordering(n)

PP = gen_points_lp(m, N, GP,Gamma)

In [ ]:
PP.shape

In [ ]:
mul = 0.1
amp = PP.T*mul

In [ ]:
shift = np.random.uniform(-0.1,0.1,size=(1,m))

In [ ]:
np.max(amp)

In [ ]:
np.max(shift)

In [ ]:
amp = amp+shift

In [ ]:
N = 64*1
dx = .1

psf = P.PsfGenerator3D(psf_shape=(N,)*3, units=(dx,)*3, na_detection=1.1, lam_detection=.5, n=1.33)

In [ ]:
max_inten = []
for a in amp:
    wf = Z.ZernikeWavefront({3:a[0], 5:a[1]}, order='ansi')
#     wf = Z.ZernikeWavefront({3:a[0], 5:a[1],}, order='ansi')
    h1 = np.fft.fftshift(psf.incoherent_psf(wf, normed=True))
    max_inten.append(np.max(h1[32]))

In [ ]:
Pmax = PP[np.argmax(np.array(:,max_inten))]
print(np.around(Pmax,3), np.max(np.array(max_inten)))

In [ ]:
Pmax = PP[:,np.argmax(np.array(max_inten))]
print(Pmax, np.max(np.array(max_inten)))

In [ ]:
np.around(Pmax*mul,5)

In [ ]:
np.around(shift,3)

In [ ]:
wf = Z.ZernikeWavefront({3:shift[0,0], 5:shift[0,1]}, order='ansi')
h1 = np.fft.fftshift(psf.incoherent_psf(wf, normed=True))
print(np.max(h1[32]))

In [ ]:
# params
lpDegree = p

In [ ]:
tr = MultiIndicesTree.MultiIndicesTree(M=m,K=n,lpDegree=lpDegree)

In [ ]:
tr.GP = GP

In [ ]:
nn, N = tr.PP.shape

In [ ]:
gamma = np.zeros([m])

In [ ]:
dds = Solvers.DDS()
D = dds.run(m,N,tr.tree, np.array(max_inten).copy(), tr.GP.copy(), gamma.copy(), 1, 1)

In [ ]:
# evaluate 
max_inten_hat = np.zeros((len(max_inten)))
for i in range(len(max_inten)):
    max_inten_hat[i] = tr.eval_lp(PP[:, i].copy(), D.copy(), m, n, N, gamma.copy(), tr.GP, tr.lpDegree, 1, 1)

In [ ]:
plt.plot(max_inten_hat,'*',label='Prediction')
plt.plot(max_inten,'.',label='GT')
plt.legend()
None;

In [ ]:
# Evaluate on random points
nx = 100
# PPx = np.random.uniform(low = Pmax-Pmax*percentage,high= Pmax+Pmax*percentage, size=(nx,m)).T
PPx = np.random.uniform(low = Pmax-Pmax*percentage,high= Pmax+Pmax*percentage, size=(nx,m)).T

In [ ]:
max_inten_hat = np.zeros(PPx.shape[0])
max_inten_hat = tr.eval_lp(PPx.copy(), D.copy(), m, n, N, gamma.copy(), tr.GP, tr.lpDegree, 1, 1)

In [ ]:
ampx = PPx.T*mul
max_intenx = []
for a in ampx:
    wf = Z.ZernikeWavefront({3:a[0], 5:a[1], 6:a[2], 7:a[3],8:a[4],9:a[5]}, order='ansi')
#     wf = Z.ZernikeWavefront({3:a[0], 5:a[1]}, order='ansi')
    h1 = np.fft.fftshift(psf.incoherent_psf(wf, normed=True))
    max_intenx.append(np.max(h1[32]))
max_intenx = np.array(max_intenx)

In [ ]:
print("Degree: %d  E: %f"%(n,np.mean(np.abs(max_intenx - max_inten_hat))))

In [ ]:
print("Degree: %d  E: %f"%(n,np.mean(np.abs(max_intenx - max_inten_hat))))

In [ ]:
wf = Z.ZernikeWavefront({3:0.00}, order='ansi')
h1 = np.fft.fftshift(psf.incoherent_psf(wf, normed=True))

In [ ]:
max_intenx

In [ ]:
max_inten_hat

In [ ]:
np.around(Pmax,3)

In [ ]:
np.around(shift,3)

In [ ]:
np.max(max_inten)